In [19]:
import json

# Ethereum With Python

## Requirements
- Solidity **Language for Smart Contract**
- See http://web3py.readthedocs.io

## Connect to Local Ethereum Sandbox

- Use Ganache (https://www.trufflesuite.com/ganache)

In [3]:
from web3 import Web3

In [4]:
# we will connect to local ganache contract
http_provider = Web3.HTTPProvider("http://localhost:7545")

In [5]:
w3 = Web3(http_provider)

In [8]:
# check connected to ethereum node
w3.isConnected()

True

In [1]:
from solc import compile_standard

In [2]:
%%bash
cat >> helloworld.sol << EOL
contract helloWorld {
    function renderHelloWorld () public pure returns (string memory) {
        return 'helloWorld';
    }
}
EOL

let's create simple contracts using `compile_standard`

In [10]:
compiled_sol = compile_standard({
    "language": "Solidity",
    "sources": {
        "helloworld.sol": {
            "content": '''
            contract helloWorld {
                function renderHelloWorld () public pure returns (string memory) {
                    return 'helloWorld';
                }
            }
            '''
        }
    },
    "settings": {
        "outputSelection": {
            "*": {
                "*": [
                    "metadata", "evm.bytecode",
                    "evm.bytecode.sourceMap"
                ]
            }
        }
    }
})

In [20]:
bytecode = compiled_sol['contracts']['helloworld.sol']['helloWorld']['evm']['bytecode']['object']
print(bytecode)

608060405234801561001057600080fd5b5061011d806100206000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063942ae0a714602d575b600080fd5b603360ab565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101560715780820151818401526020810190506058565b50505050905090810190601f168015609d5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040518060400160405280600a81526020017f68656c6c6f576f726c640000000000000000000000000000000000000000000081525090509056fea265627a7a7231582054d951b5453ab2c22bfb95679963eab03117fb0d59c85215c8646388bef1226064736f6c634300050e0032


In [28]:
## json.loads(compiled_sol['contracts']['Greeter.sol']['Greeter']['metadata'])['output']['abi']
abi = json.loads(compiled_sol['contracts']['helloworld.sol']['helloWorld']['metadata'])['output']['abi']
print(abi)

[{'constant': True, 'inputs': [], 'name': 'renderHelloWorld', 'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}], 'payable': False, 'stateMutability': 'pure', 'type': 'function'}]


In [29]:
helloworld = w3.eth.contract(abi=abi, bytecode=bytecode)

In [30]:
## tx_hash = Greeter.constructor().transact()
## it'll create some error because we did not set default account
tx_hash = helloworld.constructor().transact()

ValueError: {'message': 'from not found; is required', 'code': -32000, 'data': {'stack': 'Error: \n    at StateManager.queueTransaction (/Applications/Ganache.app/Contents/Resources/app.asar/node_modules/ganache-core/lib/statemanager.js:314:14)\n    at GethApiDouble.eth_sendTransaction (/Applications/Ganache.app/Contents/Resources/app.asar/node_modules/ganache-core/lib/subproviders/geth_api_double.js:322:14)\n    at GethApiDouble.handleRequest (/Applications/Ganache.app/Contents/Resources/app.asar/node_modules/ganache-core/lib/subproviders/geth_api_double.js:108:10)\n    at next (/Applications/Ganache.app/Contents/Resources/app.asar/node_modules/ganache-core/node_modules/web3-provider-engine/index.js:136:18)\n    at GethDefaults.handleRequest (/Applications/Ganache.app/Contents/Resources/app.asar/node_modules/ganache-core/lib/subproviders/gethdefaults.js:15:12)\n    at next (/Applications/Ganache.app/Contents/Resources/app.asar/node_modules/ganache-core/node_modules/web3-provider-engin

In [31]:
## set default account
w3.eth.defaultAccount = w3.eth.accounts[0]

In [32]:
tx_hash = helloworld.constructor().transact()

In [33]:
tx_hash

HexBytes('0xb9bf1d55e31b2e050d2679dd9007d29a1c81014b6065c85935aeb1d9cd215889')

In [34]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [36]:
## the contract is created and mined as block no. 2
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xb9bf1d55e31b2e050d2679dd9007d29a1c81014b6065c85935aeb1d9cd215889'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x16084efc493337755451d95539d6f229fc1eb70b3e658865f7843a9bbd165023'),
 'blockNumber': 2,
 'from': '0x510670dc3b22aa1d6234f426d25c0f834471a820',
 'to': None,
 'gasUsed': 129485,
 'cumulativeGasUsed': 129485,
 'contractAddress': '0xEEeB1F6d4E4fa03CD5D31cdB7c9dA2519C05168d',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [37]:
# greeter = w3.eth.contract(
#     address=tx_receipt.contractAddress,
#     abi=abi
# )
helloWorldFromContract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

In [39]:
## greeter.functions.greet().call()
helloWorldFromContract.functions.renderHelloWorld().call()

'helloWorld'